In [224]:
import os
import shutil
import string
from enum import Enum
from collections import Counter
import random
import numpy as np
from copy import copy

In [ ]:
!wget https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip 

--2022-05-08 12:18:41--  https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/k23enjvr3fb40o5/corpora.zip [following]
--2022-05-08 12:18:41--  https://www.dropbox.com/s/raw/k23enjvr3fb40o5/corpora.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3d03e0483cd87a2a189421f912.dl.dropboxusercontent.com/cd/0/inline/Bk7HqdpPfElGFdvECNM4rVN70uqCyKvDM6tnuRpYt1TzIKfN7VoOSCDe9xUTi7kyfre8E_HboPYyoAmKu5eevmMGOOE378Lhug20z03u4fqU3pC4JGJTHrR-qsVr1aDmgwE5VJghkWTqqz8HdI2jkWGduKPz95bsve_7r6KhGuAZ8w/file# [following]
--2022-05-08 12:18:41--  https://uc3d03e0483cd87a2a189421f912.dl.dropboxusercontent.com/cd/0/inline/Bk7HqdpPfElGFdvECNM4rVN70uqCyKvDM6tnuRpYt1TzIKfN7VoOSCDe9xUTi7kyfre8E

In [ ]:
shutil.unpack_archive("corpora.zip", "./data")

In [399]:
with open(os.path.join('data', 'AnnaKarenina.txt'), 'r') as f:
    anna_karenina = f.read()
with open(os.path.join('data', 'WarAndPeace.txt'), 'r') as f:
    war_and_peace = f.read()

with open(os.path.join('data', 'WarAndPeaceEng.txt'), 'r') as f:
    war_and_peace_eng = f.read()

train_text_en = war_and_peace_eng
train_text_ru = anna_karenina + war_and_peace

In [400]:
class Language(Enum):
  RU = 1
  EN = 2

## Basic

In [401]:
class FrequencyTransformer:
   ENG_ALPHABET = string.ascii_lowercase + ' '
   RU_ALPHABET = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя' + ' '
   
   def __init__(self):
        self.corpus = None
        self.n_gram_size = None
        self.freq_dict = None

   def fit(self, text: str, language: Language, n_gram_size: int = 1):
        self.corpus = FrequencyTransformer.ENG_ALPHABET if language == Language.EN else FrequencyTransformer.RU_ALPHABET
        self.n_gram_size = n_gram_size
        temp = self._preprocess(text)
        n_grams = self._get_ngrams(temp)
        self.freq_dict = self._get_counter_dict(n_grams)

        return self
        
   def decode(self, text: str) -> str:
        temp =  self._preprocess(text)
        temp =  temp  + ' ' * (len(temp) % self.n_gram_size)
        n_grams = self._get_ngrams(temp)
        encoded_dict = self._get_counter_dict(n_grams)
        mapper = {}
        for d1, d2 in zip(encoded_dict.items(), self.freq_dict.items()):
            ng1 = d1[0]
            ng2 = d2[0] 
            mapper[ng1] = ng2
        return ''.join([mapper[ng] for ng in n_grams])[:len(temp)]

   def _get_ngrams(self, text):
        return [text[i:i+self.n_gram_size] for i in range(len(text) - self.n_gram_size + 1)]
  
   def _get_counter_dict(self, ngrams):
        return {k: v for k, v in sorted(Counter(ngrams).items(), key=lambda item: item[1], reverse=True)}

   def _preprocess(self, text: str):
        return ''.join([c for c in text.lower() if c in self.corpus])


In [402]:
class Encoder:
   ENG_ALPHABET = string.ascii_lowercase + ' '
   RU_ALPHABET = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя' + ' '
   
   def __init__(self):
        self.alphabet = None

   def fit(self, language: Language, filter_only=False):
        self.alphabet = FrequencyTransformer.ENG_ALPHABET if language == Language.EN else FrequencyTransformer.RU_ALPHABET
        self.filter_only = filter_only
        
        return self


   def encode(self, text: str) -> str:
        temp =  self._preprocess(text)
        alphabet_shuffled = ''.join(random.sample(self.alphabet,len(self.alphabet)))
        map = {}
        for c1, c2 in zip(self.alphabet, alphabet_shuffled):
          map[c1] = c2 if not self.filter_only else c1
        return ''.join([map[c] for c in temp])

   def _preprocess(self, text: str):
        return ''.join([c for c in text.lower() if c in self.alphabet]) 

In [403]:
text = '''
Глаза Степана Аркадьича весело заблестели, и он задумался, улыбаясь. «Да, хорошо было, очень хорошо. Много еще что-то там было отличного, да не скажешь словами и мыслями даже наяву не выразишь». И, заметив полосу света, пробившуюся сбоку одной из суконных стор, он весело скинул ноги с дивана, отыскал ими шитые женой (подарок ко дню рождения в прошлом году), обделанные в золотистый сафьян туфли и по старой, девятилетней привычке, не вставая, потянулся рукой к тому месту, где в спальне у него висел халат. И тут он вспомнил вдруг, как и почему он спит не в спальне жены, а в кабинете; улыбка исчезла с его лица, он сморщил лоб.
'''
text = Encoder().fit(Language.RU, filter_only=True).encode(text)
text

'глаза степана аркадьича весело заблестели и он задумался улыбаясь да хорошо было очень хорошо много еще чтото там было отличного да не скажешь словами и мыслями даже наяву не выразишь и заметив полосу света пробившуюся сбоку одной из суконных стор он весело скинул ноги с дивана отыскал ими шитые женой подарок ко дню рождения в прошлом году обделанные в золотистый сафьян туфли и по старой девятилетней привычке не вставая потянулся рукой к тому месту где в спальне у него висел халат и тут он вспомнил вдруг как и почему он спит не в спальне жены а в кабинете улыбка исчезла с его лица он сморщил лоб'

In [405]:
def accuracy(ground_truth : str, result : str):
    return sum([i==j for (i, j) in zip(ground_truth, result)])/len(ground_truth)

In [406]:
encoder = Encoder().fit(Language.RU)

text_encoded = encoder.encode(text)
text_encoded

'вькюкцлбядкёкцкоукшэыскцтяляьйцюкжьялбяьыцыцйёцюкшфакьлгцфьржкглэцшкцийойъйцжрьйцйсяёэцийойъйцаёйвйцяпяцсбйбйцбкацжрьйцйбьысёйвйцшкцёяцлукзяъэцльйткаыцыцарльгаыцшкзяцёкгтфцёяцтрокюыъэцыцюкаябытцдйьйлфцлтябкцдойжытъфхлгцлжйуфцйшёйецыюцлфуйёёрицлбйоцйёцтяляьйцлуыёфьцёйвыцлцшыткёкцйбрлукьцыаыцъыбряцзяёйецдйшкойуцуйцшёхцойзшяёыгцтцдойъьйацвйшфцйжшяькёёряцтцюйьйбылбрецлк эгёцбф ьыцыцдйцлбкойецшятгбыьябёяецдоытрсуяцёяцтлбкткгцдйбгёфьлгцофуйецуцбйафцаялбфцвшяцтцлдкьэёяцфцёявйцтыляьцикькбцыцбфбцйёцтлдйаёыьцтшофвцукуцыцдйсяафцйёцлдыбцёяцтцлдкьэёяцзяёрцкцтцукжыёябяцфьржукцылсяюькцлцявйцьымкцйёцлайопыьцьйж'

### Unogramm

In [407]:
transformer = FrequencyTransformer().fit(train_text_ru, Language.RU, 1)

In [408]:
decoded = transformer.decode(text_encoded)
decoded

'ысебе тлапене емкедзиче ватасо бегсатласи и он бедруесть рсягеьтз де хоможо гясо очанз хоможо уноыо аюа члоло леу гясо олсичноыо де на ткешажз тсовеуи и уятсьуи деша неьвр на вямебижз и беуалив посотр твале пмогивжрэть тгокр одной иб трконнях тлом он ватасо ткинрс ноыи т дивене оляткес иуи жиляа шаной подемок ко днэ мошданиь в пможсоу ыодр огдасенняа в босолитляй тецзьн лрцси и по тлемой давьлисалнай пмивячка на втлевеь польнрсть мркой к лоур уатлр ыда в тпесзна р наыо витас хесел и лрл он втпоунис вдмры кек и почаур он тпил на в тпесзна шаня е в кегинала рсягке итчабсе т аыо сище он туомюис сог'

In [409]:
accuracy(text, decoded)

0.5132890365448505

### Bigramm

In [410]:
transformer = FrequencyTransformer().fit(train_text_ru, Language.RU, 2)

In [411]:
decoded = transformer.decode(text_encoded)
decoded

'едлабыра но нав со млива н рреиенежеоеисем ж не зай н он пи енраседи гй нав он д сто с вал кенражеиншехокаестаовсквотиитевтечапем норо номдо бст еаси егльоб пи  ввслеотадм омдо бст еаси аню  тк нии  лнябоа  ябеу л у и просогмоегльоб пи  вл ее демзн тк нии норо нь  оа о ринече чожойм о ав псвилогде сто саниз аавдуляде снороче ча ь ватеияичя ь  оа е сясаспбыуддройм то сенраогнылот олго ипось пшис я о икзалоос н иатстетннолокидужектаово днглмеочя  водми ттькототыкро с х веалимакку хо нау  бси вал ке зай н он пи о риих экиво  ь  тк ят со ивножаолиллива н вл ы  аринека  тожиде сийтуэты муа ма чот тть'

In [412]:
accuracy(text, decoded)

0.06976744186046512

## MCMC

- pick up a random current state.
- swap two random letters in the current state -> new state proposal 
- calculate the score of the current state and the proposed state
- if score is better, move to proposed state
- else accept new state with prob (score of proposed state / score of current state)
- repeat

In [413]:
class MCMCDecoder:
   ENG_ALPHABET = string.ascii_lowercase + ' '
   RU_ALPHABET = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя' + ' '
   EPS = 0.0000000001
   
   def __init__(self):
        self.corpus = None
        self.n_gram_size = None
        self.freq_dict = None
        self.n_iter = None

   def fit(self, text: str, language: Language, n_gram_size: int = 1, n_iter=10000):
        self.corpus = MCMCDecoder.ENG_ALPHABET if language == Language.EN else MCMCDecoder.RU_ALPHABET
        self.n_gram_size = n_gram_size
        temp = self._preprocess(text)
        n_grams = self._get_ngrams(temp)
        self.freq_dict = self._get_counter_dict(n_grams)
        self.n_iter = n_iter

        return self

   def decode(self, text: str) -> str:
       result = None
       current = copy(text)
       cur_score = self._score_log(current)
       best_score = None
       for it in range(self.n_iter):
         text_candidate = self._get_new_state(current)
         candidate_score = self._score_log(text_candidate)
         if best_score is None or cur_score > best_score or self._should_accept(candidate_score, cur_score):
           current = text_candidate
           cur_score = candidate_score
           if best_score is None or cur_score > best_score:
             best_score = copy(cur_score)
             result = copy(current)
       return result
                


   def _get_new_state(self, text : str) -> str:
     temp = copy(text)
     letters = list(temp)
     swaps = np.random.choice(list(self.corpus), 2, replace=False)
     result = []
     for letter in letters:
       if letter == swaps[0]:
         result += swaps[1]
       elif letter == swaps[1]:
         result += swaps[0]
       else:
         result += letter

     return ''.join(result)

        
   def _score_log(self, text: str):
        temp =  text + ' ' * (len(text) % self.n_gram_size)
        n_grams = self._get_ngrams(temp)
        proposed_count_dict = self._get_counter_dict(n_grams)
        return np.sum([count * np.log(self.freq_dict.get(ngram,  MCMCDecoder.EPS)) 
                       for ngram, count in proposed_count_dict.items()])
        
   def _should_accept(self, candidate_score_log, cur_score_log):
     return  np.random.rand() < np.exp(candidate_score_log - cur_score_log)
    
   def _get_ngrams(self, text):
        return [text[i:i+self.n_gram_size] for i in range(len(text) - self.n_gram_size + 1)]
  
   def _get_counter_dict(self, ngrams):
        return {k: v for k, v in sorted(Counter(ngrams).items(), key=lambda item: item[1], reverse=True)}

   def _preprocess(self, text: str):
        return ''.join([c for c in text.lower() if c in self.corpus]) 
 

In [414]:
transformer = MCMCDecoder().fit(train_text_ru, Language.RU, 2, 40000)

In [424]:
decoded = transformer.decode(text_encoded)
decoded

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: RuntimeWarning: overflow encountered in exp


'глаза степана аркадьича весело заблестели и он задумался улыбаясь да хорошо было очень хорошо много еще чтото там было отличного да не скажешь словами и мыслями даже наяву не выразишь и заметив полосу света пробившуюся сбоку одной из суконных стор он весело скинул ноги с дивана отыскал ими шитые женой подарок ко дню рождения в прошлом году обделанные в золотистый сафьян туфли и по старой девятилетней привычке не вставая потянулся рукой к тому месту где в спальне у него висел халат и тут он вспомнил вдруг как и почему он спит не в спальне жены а в кабинете улыбка исчезла с его лица он сморщил лоб'

In [425]:
accuracy(text, decoded)

1.0

## Secret word

In [426]:
secret = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'
secret_letters = list(set(secret))

In [427]:
EN_ALPHABET = string.ascii_lowercase
RU_ALPHABET =  'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

print(len(EN_ALPHABET))
print(len(RU_ALPHABET))
print(len(secret_letters))

26
33
28


In [428]:
ru_map = {k:v for k, v in zip(secret_letters, RU_ALPHABET)}

In [429]:
candidate_ru = ''.join([ru_map[x] for x in secret])

In [430]:
transformer = MCMCDecoder().fit(train_text_ru, Language.RU, 2, 80000)

In [431]:
# need some luck to get a good result
for _ in range(5):
  decoded = transformer.decode(candidate_ru)
  print(decoded)

если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж
если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж
оилаврявражатовн спельнязвалавы ктавн спельнязвтомитвувчт й ви  бгонадвм т сязвлойм выс катетьвим соовриой врявриовижолелавысеральн вавы лукатовпемиапельнязвбеллвшевы иложноовкотрост овшеженаовмусиевх тдвм нокн вдвнакой внов богещ
натерждржелеонри зяствидпретерь моери зяствидпронкаоруршо ч ра  быниегрк о здпртнчк рьз меосоврак зннржанч рждржанралнтстерьзсжетви рерь тумеонряскаеяствидпрбсттрюсрь атнлиннрмножнзо нрюслсиенркузасрх огрк инми ргриемнч ринр бнысё
если вы вимите нордальный или почти нордальный текст у этого сообщения котор